In [ ]:
pip install transformers datasets

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import ViTModel, ViTConfig, BertTokenizer, ViTImageProcessor
from datasets import load_dataset
from PIL import Image
from datasets import DatasetDict

import numpy as np

In [ ]:
# Load the dataset
dataset = load_dataset("tomytjandra/h-and-m-fashion-caption-12k")  # Replace with your dataset path or identifier

# Initialize the tokenizer (you can choose a different tokenizer if preferred)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define image transformations
# image_transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # ViT typically expects 224x224 images
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet statistics
#                          std=[0.229, 0.224, 0.225]),
# ])

processor = ViTImageProcessor().from_pretrained('google/vit-base-patch16-224-in21k')

# Preprocessing function
def preprocess_function(examples):
    # Process images
    images = [image.convert("RGB") for image in examples['image']]
    encoding = processor(images=images)
    examples['pixel_values'] = encoding['pixel_values']

    # Tokenize captions
    captions = examples['text']
    encoding = tokenizer(captions, padding='max_length', truncation=True, max_length=224)
    examples['input_ids'] = encoding['input_ids']
    examples['attention_mask'] = encoding['attention_mask']

    return examples

# Optional: Verify the splits
processed_ds = dataset.map(preprocess_function, batched=True, batch_size=100, remove_columns=['text', 'image'])

processed_ds.set_format(
    type='torch',
    columns=['pixel_values', 'input_ids', 'attention_mask']
)

# Step 1: Split into train_val and test
train_val_split = processed_ds['train'].train_test_split(test_size=1250, seed=42)  # 10% for test
train_val = train_val_split['train']
test = train_val_split['test']

# Step 2: Split train_val into train and validation
train_validation_split = train_val.train_test_split(test_size=1250, seed=42)
train = train_validation_split['train']
validation = train_validation_split['test']

# Step 3: Create a new DatasetDict with the splits
new_dataset = DatasetDict({
    'train': train,
    'validation': validation,
    'test': test
})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Define training parameters
batch_size = 32

# Create DataLoader for training
train_loader = DataLoader(new_dataset['train'], batch_size=batch_size, shuffle=True)
val_loader = DataLoader(new_dataset['validation'], batch_size=batch_size, shuffle=True)

In [ ]:
class ImageCaptioningModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, decoder_layers, decoder_heads, decoder_ffn_dim, max_seq_length=224):
        super(ImageCaptioningModel, self).__init__() #initialize from parent .init()

        # Encoder: Vision Transformer (ViT)
        vit_config = ViTConfig.from_pretrained('google/vit-base-patch16-224-in21k')
        vit_config.num_hidden_layers = 6  # Reduce the number of layers to 4-6
        self.encoder = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k', config=vit_config)

        # Decoder: Transformer
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=decoder_heads, dim_feedforward=decoder_ffn_dim)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=decoder_layers)

        # Embedding for input tokens
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_seq_length, embed_dim)) # ??????????????

        # Final linear layer to generate vocabulary scores
        self.output_linear = nn.Linear(embed_dim, vocab_size)

        # Projection to match dimensions
        self.encoder_proj = nn.Linear(vit_config.hidden_size, embed_dim)

    def forward(self, pixel_values, input_ids, attention_mask):
        # Encoder
        encoder_outputs = self.encoder(pixel_values=pixel_values)
        encoder_hidden_states = encoder_outputs.last_hidden_state  # (batch_size, num_patches + 1, hidden_size)
        # Project encoder outputs to embed_dim
        encoder_proj = self.encoder_proj(encoder_hidden_states)  # (batch_size, seq_len, embed_dim)
        encoder_proj = encoder_proj.permute(1, 0, 2)  # (seq_len, batch_size, embed_dim)

        # Decoder
        embeddings = self.token_embedding(input_ids) + self.positional_encoding[:, :input_ids.size(1), :]
        embeddings = embeddings.permute(1, 0, 2)  # (seq_len, batch_size, embed_dim)


        decoder_outputs = self.decoder(embeddings, encoder_proj, tgt_key_padding_mask=~attention_mask.bool())
        decoder_outputs = decoder_outputs.permute(1, 0, 2)  # (batch_size, seq_len, embed_dim)

        outputs = self.output_linear(decoder_outputs)  # (batch_size, seq_len, vocab_size)

        return outputs

In [ ]:
# Define vocabulary size and other hyperparameters
vocab_size = tokenizer.vocab_size
embed_dim = 512
decoder_layers = 6  # 4-6 layers as per requirement
decoder_heads = 8
decoder_ffn_dim = 2048
max_seq_length = 224

# Initialize the model
model = ImageCaptioningModel(vocab_size, embed_dim, decoder_layers, decoder_heads, decoder_ffn_dim, max_seq_length)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Define separate learning rates
learning_rate_encoder = 1e-5  # Lower learning rate for pre-trained encoder
learning_rate_decoder = 1e-4  # Higher learning rate for decoder
# Create parameter groups
optimizer = optim.AdamW([
    {'params': model.encoder.parameters(), 'lr': learning_rate_encoder},
    {'params': model.decoder.parameters(), 'lr': learning_rate_decoder},
    {'params': model.token_embedding.parameters(), 'lr': learning_rate_decoder},
    {'params': model.encoder_proj.parameters(), 'lr': learning_rate_decoder},
    {'params': model.output_linear.parameters(), 'lr': learning_rate_decoder}
], betas=(0.9, 0.98), eps=1e-9)

# Initialize the scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                 factor=0.5, patience=2,
                                                 verbose=True, min_lr=1e-6)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['encoder.layer.10.attention.attention.key.bias', 'encoder.layer.10.attention.attention.key.weight', 'encoder.layer.10.attention.attention.query.bias', 'encoder.layer.10.attention.attention.query.weight', 'encoder.layer.10.attention.attention.value.bias', 'encoder.layer.10.attention.attention.value.weight', 'encoder.layer.10.attention.output.dense.bias', 'encoder.layer.10.attention.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.layernorm_after.bias', 'encoder.layer.10.layernorm_after.weight', 'encoder.layer.10.layernorm_before.bias', 'encoder.layer.10.layernorm_before.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.attention.attention.key.bias', 'encoder.layer.11.attention.attention.key.weight', 'encoder.layer.11.attention.attention.q

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    epoch_val_loss = 0
    for batch in train_loader:
        pixel_values = batch['pixel_values'].to(device)  # (batch_size, 3, 224, 224)
        input_ids = batch['input_ids'].to(device)        # (batch_size, seq_length)
        attention_mask = batch['attention_mask'].to(device)  # (batch_size, seq_length)

        # Shift input_ids and create labels
        # Typically, input_ids are shifted right for the decoder input
        # Labels are the actual tokens to predict
        labels = input_ids[:, 1:].contiguous()
        decoder_input_ids = input_ids[:, :-1].contiguous()
        decoder_attention_mask = attention_mask[:, :-1].contiguous()

        optimizer.zero_grad()

        outputs = model(pixel_values, decoder_input_ids, decoder_attention_mask)
        # outputs: (batch_size, seq_length -1, vocab_size)

        loss = criterion(outputs.view(-1, vocab_size), labels.view(-1))
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)

    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch['pixel_values'].to(device)  # (batch_size, 3, 224, 224)
            input_ids = batch['input_ids'].to(device)        # (batch_size, seq_length)
            attention_mask = batch['attention_mask'].to(device)  # (batch_size, seq_length)

            labels = input_ids[:, 1:].contiguous()
            decoder_input_ids = input_ids[:, :-1].contiguous()
            decoder_attention_mask = attention_mask[:, :-1].contiguous()

            outputs = model(pixel_values, decoder_input_ids, decoder_attention_mask)

            loss = criterion(outputs.view(-1, vocab_size), labels.view(-1))

            epoch_val_loss += loss.item()

    avg_val_loss = epoch_val_loss / len(val_loader)

    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, val_loss: {avg_val_loss:.4f}, lr: {scheduler.get_last_lr()}")

Epoch 1/10, Loss: 3.1553, val_loss: 1.5552, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 2/10, Loss: 1.3076, val_loss: 1.1033, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 3/10, Loss: 0.9947, val_loss: 0.9175, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 4/10, Loss: 0.8530, val_loss: 0.8573, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 5/10, Loss: 0.7621, val_loss: 0.7923, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 6/10, Loss: 0.6940, val_loss: 0.7607, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 7/10, Loss: 0.6404, val_loss: 0.7328, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 8/10, Loss: 0.5951, val_loss: 0.7264, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 9/10, Loss: 0.5548, val_loss: 0.7108, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]
Epoch 10/10, Loss: 0.5187, val_loss: 0.7305, lr: [1e-05, 0.0001, 0.0001, 0.0001, 0.0001]


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/VIT+decoder/VIT_decoder.pth')

# model = MyModel()
# model.load_state_dict(torch.load('model_state.pth'))
# model.eval()  # Set the model to evaluation mode

In [ ]:
#inference
#tgt_mask nad what it mean to attent only to previous tokens (look from MLCV lectures)
#save on each epoch or each N epochs
#